In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import json
import sys
import re
import logging
import os
import re
from pathlib import Path
from functools import partial, cache, cached_property
from itertools import chain
from inspect import signature
from uuid import uuid4

import blpapi
import ipywidgets as w
import ipyvuetify as v
import numpy as np
import pandas as pd
import QuantLib as ql
import talib as ta 
from IPython.display import display
from ipydatagrid import DataGrid, TextRenderer, BarRenderer, Expr, VegaExpr
from bqplot import LinearScale, ColorScale, OrdinalColorScale, OrdinalScale
from pandas.tseries.offsets import BDay
from plotly import graph_objects as go
from plotly.subplots import make_subplots
from termcolor import colored
# from bhsysapi import BHSysApi

from techi import api as ti

from gioutils import ezutils as ez
from gioutils.ezutils import bh, bhs
from gioutils.blpw import BlpQuery
from gioutils.gui.pandas import millify
from gioutils.utils import (
    RepeatedTimer, 
    parse_swap_tenor_expr, 
    parse_offset,
    today, 
    get_next_n, 
    get_n_from_code,
    date_from_offset, 
    get_bday,
    get_bbg_fut_chain_ticker
)

from api.data.base import data_path, radar_path
from api.gui.params import bbg_params, spot_params, fwd_params
from api.instruments.swap import RealSwap, RollingSwap, Fly, Spread
from api.instruments.utils import parse_dates, parse_number, ez_bump
from api.instruments.strategies import FutureStrategy
from api.instruments.signals import MACDHistCrossover, ZScoreThreshold, EnsambleSignal
from api.instruments.cache import data_cache
from api.instruments.future import FutureChain, FutureSpread, FutureFly
from api.gui.theme import bg_color, grid_style
from api.gui.pandas import DFOutput, negative_red, custom_styler_row_line, DataGridGio
from api.instruments import config as c
from api.instruments.signals2 import BaseSignal, MACDCustomSignal, get_crossover_signal
from api.instruments.strategies2 import MACDCustomStrategy, get_next_signal

In [3]:
pd.options.plotting.backend = "plotly"
pd.options.display.float_format = millify
c.USE_T_COST = True

In [4]:
bq = BlpQuery(timeout=50000).start()
# ba = BHSysApi(password="A&3jg5Aj")

In [5]:
instrument = FutureChain(
    'rx1', 
    notional=1000, 
    bq=bq, 
    start_hist='10y'
)

In [6]:
signal_threshold_buy = -0.7
signal_threshold_sell = 0.7
slope_threshold_buy = 0
slope_threshold_sell = 0

In [7]:
signal = MACDCustomSignal(
    # instrument=instrument,
    signal_threshold_buy=-signal_threshold_buy,
    signal_threshold_sell=signal_threshold_sell,
    slope_threshold_buy=slope_threshold_buy,
    slope_threshold_sell=slope_threshold_sell,
)

In [8]:
strategy = MACDCustomStrategy(
    instrument=instrument,
    signal=signal,
    time_barrier=4,  # range(4, 7 + 1)
    q=0.4,  # range(0.4, 0.9 + q_step, q_step)
    k=2,  # range(2, 3 + k_step, k_step)
    crossover_days=1,
    # verbose=True,
    # start_hist='1y',
)
self = strategy

In [13]:
%%timeit
data = self.data

269 ms ± 2.11 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


# self._summary.to_clipboard()

In [ ]:
self.run()

In [ ]:
self._data['pv'].plot()

In [ ]:
run_dates = self.get_date_range(data)

In [ ]:
run_dates

In [ ]:
signals = signals.loc[run_dates[0]:run_dates[1]]

In [ ]:
signals

In [ ]:
data = self.data

In [ ]:
data

In [ ]:
summary = self._summary

In [ ]:
((summary['exit_price'] - summary['entry_price']) * summary['signal']).sum()

In [ ]:
data['pv']

In [ ]:
data['pnl'].sum()

In [ ]:
instrument.notional

In [ ]:
data['quote_chg'].sum() * 1000 * 1000

In [ ]:
data.to_clipboard()


In [ ]:
summary

In [ ]:
summary = strategy._summary
summary.to_clipboard()

In [ ]:
pnl = strategy._info

In [ ]:
s = strategy

In [ ]:
pd.concat([s._data, s._info], axis=1)

In [ ]:
instrument.point_value

In [ ]:
pnl['pnl'].sum()

In [ ]:
for index, row in summary.iterrows():
    pnl_trade = pnl.query(f"trade == {row['trade']}")
    pnl_sum = pnl_trade['pnl'].sum()
    if row['pnl'] != pnl_sum:
        print('ERROR', row['trade'])
        break
    else:
        print('OK') 

In [ ]:
pnl_trade

In [ ]:
((summary['exit_price'] - summary['entry_price']) * summary['signal']).sum()

In [ ]:
pd.DataFrame(pnl)['date'].value_counts().sort_index()

In [ ]:
p = pd.concat([pd.DataFrame(pnl).set_index('date'), instrument.quote], axis=1)

In [ ]:
p['pnl'].sum()

In [ ]:
signals = signal.signals
data = signal.data

In [ ]:
strategy._data.to_clipboard()

In [ ]:
data.to_clipboard()

In [ ]:
data['crossover'] = get_crossover_signal(data['macdhist'], threshold=False, fill=0)
data['quote'] = instrument.quote

In [ ]:
def get_next_signal(signals, index=None, last=False):
    ss = signals.copy()
    if last:
        ss = ss.sort_index(ascending=False)
    if index:
        ss.iloc[:index + 1] = 0
    signal_date = (ss.fillna(0) != 0).idxmax()
    signal_index = ss.index.get_loc(signal_date)
    return ss.loc[signal_date], signal_date, signal_index

In [ ]:
def get_base_stop(quote, periods=7, mult=2):
    return quote[-periods:].diff().std() * mult

In [ ]:
def barrier_hit(quote, stop_loss, take_profit):
    pass

In [ ]:
time_barrier = 4  # range(4, 7 + 1)
q = 0.4  # range(0.4, 0.9 + q_step, q_step)
k = 2  # range(2, 3 + k_step, k_step)
p = k * q
crossover_days = 1  #  range(0, 2 + 1)

In [ ]:
i = 0
debug = False
trade_count = 0
end_date = get_next_signal(signals, last=True)[1]
end = len(data)
summary = []
pnl = []
while True:
    current_signal, signal_date, signal_index = get_next_signal(signals, index=i)
    crossover = False
    crossover_date = None
    crossover_index = None
    change_after_crossover = None 
    # current_signal = data['signal'].iloc[signal_index]
    start_price = data['quote'].iloc[signal_index] 
    entry_price = start_price + current_signal * instrument.ticks_to_points(instrument.t_cost)
    base_stop_loss = get_base_stop(data['quote'].iloc[:signal_index + 1])
    base_take_profit = base_stop_loss
    stop_loss = base_stop_loss
    take_profit = base_take_profit
    first_cross = True
    trade_count += 1
    
    pnl.append(
        {
            'date': signal_date,
            'trade': trade_count,
            # 'quote': start_price,
            'pnl': (start_price - entry_price) * current_signal,
            'stop_loss': stop_loss,
            'take_profit': take_profit
        }
    )
    
    for j, i in enumerate(range(signal_index + 1, end), 1):
        data_ = data[:i + 1]
        current_price = data_['quote'].iloc[-1]
        current_date = data_.index[-1]
        pnl_ltd = (current_price - start_price) * current_signal
        current_pnl = (current_price - data_['quote'].iloc[-2]) * current_signal
   
        if j == 1:
            text = f'START {signal_date:%Y%b%d} trading signal {current_signal} @ {start_price}'
            print(colored(text, attrs=['bold']))

#         if signal_date == pd.Timestamp(2022, 9, 30):
#             debug = True
#             break
            
#         if current_date == pd.Timestamp(2022, 10, 18):
#             debug = True
#             break
        
        if j < time_barrier and pnl_ltd <= - stop_loss:
            text = f'\t{current_date:%Y%b%d} stop loss {stop_loss:.4f} hit before the crossing time barrier: exit'
            print(colored(text, color='red', attrs=['bold']))
            break
        
        if np.abs(data_.iloc[-1]['crossover']) == 1 and first_cross:
            crossover = True
            first_cross = False
            crossover_date = data.index[i]
            crossover_index = i
        
        if j >= time_barrier and not crossover:
            text = f'\t{current_date:%Y%b%d} time barrier hit'
            
            if pnl_ltd >= 0:
                text = f'{text}: profit {pnl_ltd:.4f}'
                print(colored(text, color='green', attrs=['bold']))
            else:
                text = f'{text}: loss {pnl_ltd:.4f}'
                print(colored(text, color='red', attrs=['bold']))
            break
        elif crossover:
            # text = f'\t{current_date:%Y%b%d} crossover'
            # print(text)
            
            change_after_crossover = (
                (
                    data_['quote'].iloc[crossover_index:crossover_index + crossover_days + 1] - 
                    data_['quote'].iloc[signal_index]
                ) * current_signal
            ).max()
            if change_after_crossover <= 0:  # here I stop if after entering there is a cross but it's against my trade
                text = f'\tchange since crossover < 0 loss {pnl_ltd:.4f}: exit'
                print(colored(text, color='red', attrs=['bold']))
                break
            
            stop_loss = q * change_after_crossover
            take_profit = k * q * change_after_crossover
            if (pnl_ltd <= - stop_loss):
                text = f'\tcrossover: stop loss since {stop_loss:.4f} hit: exit'
                print(colored(text, color='red', attrs=['bold']))
                break
            
            if (pnl_ltd >= take_profit):
                text = f'\tcrossover: take profit since {take_profit:.4f} hit: exit'
                print(colored(text, color='green', attrs=['bold']))
                break
                
        pnl.append(
            {
                'date': current_date,
                'trade': trade_count,
                # 'quote': current_price,
                'pnl': current_pnl,
                'stop_loss': stop_loss,
                'take_profit': take_profit
            }
        )
    if debug:
        break
    exit_price = current_price - current_signal * instrument.ticks_to_points(instrument.t_cost)
    
    last_pnl = {
        'date': current_date,
        'trade': trade_count,
        # 'quote': current_price,
        'pnl': (exit_price - data_['quote'].iloc[-2]) * current_signal,
        'stop_loss': stop_loss,
        'take_profit': take_profit,  
    }
    
    if pnl[-1]['date'] == current_date:
        pnl[-1] = last_pnl
    else:
        pnl.append(last_pnl)
        
    summary.append(
        {
            'trade': trade_count,
            'signal': current_signal,
            'entry_date': signal_date,
            'exit_date': current_date, 
            'entry_price': entry_price, 
            'exit_price': exit_price,
            'start_price': start_price,
            'end_price': current_price,
            'pnl': (exit_price - entry_price) * current_signal
        }
    )
    if signal_date == end_date:
        break

In [ ]:
summary = pd.DataFrame(summary)
summary

In [ ]:
summary.to_clipboard()

In [ ]:
pnl = pd.DataFrame(pnl)

In [ ]:
strategy._info.to_clipboard()

In [ ]:
pnl['pnl'].sum()

In [ ]:
pnl.to_clipboard()

In [ ]:
for index, row in summary.iterrows():
    pnl_trade = pnl.query(f"trade == {row['trade']}")
    pnl_sum = pnl_trade['pnl'].sum()
    if row['pnl'] != pnl_sum:
        print('ERROR', row['trade'])
        break
    else:
        print('OK') 

In [ ]:
pnl_trade

In [ ]:
((summary['exit_price'] - summary['entry_price']) * summary['signal']).sum()

In [ ]:
pd.DataFrame(pnl)['date'].value_counts().sort_index()

In [ ]:
p = pd.concat([pd.DataFrame(pnl).set_index('date'), instrument.quote], axis=1)

In [ ]:
p['pnl'].sum()

In [ ]:
data_[-7:]

In [ ]:
data.to_clipboard()

In [ ]:
current_signal, signal_date, signal_index

In [ ]:
get_next_signal(signals, index=i)

In [ ]:
res[0]['exit_price'] - res[0]['entry_price'] 

In [ ]:
res[0]['end_price'] - res[0]['start_price'] 

In [ ]:
res.to_clipboard()

In [ ]:
pd.DataFrame(np.random.normal(size=(10, 10)))

In [ ]:
res[35:]

In [ ]:
res.to_clipboard()

In [ ]:
bq.bdib(
    'rx1 Comdty', 
    event_type='TRADE',
    start_datetime='2020-01-01',
    end_datetime='2022-12-01',
    interval=5,
)

In [ ]:
df = pd.concat(
    [
        self.quote,
        ti.EMA(self.quote, 12),
        ti.EMA(self.quote, 26),
        ma.ti,
        ma.line.diff(),
    ], 
    axis=1
)

In [ ]:
es = EnsambleSignal(
    instrument=self,
    signals=[
        MACDSignal(),
        ZScoreThreshold()
    ],
    method='sum2'
)


In [ ]:
c = ((self.quote - self.quote.rolling(60).mean()) / self.quote.rolling(60).std()).abs() 

In [ ]:
c.iloc[:c.index.get_loc(c.first_valid_index())]

In [ ]:
self.quote.zscore(window=60, rolling=True)[:60]

In [ ]:
sum

In [ ]:
es.signals.first_valid_index()

In [ ]:
params = {**signature(pd.DataFrame.ewm).parameters, **signature(pd.DataFrame.rolling).parameters}

In [ ]:
kwargs = {}

In [ ]:
fn_kwargs = {}
for p, v in params.items():
    if p != 'self':
        fn_kwargs[p] = kwargs.pop(p, v.default)

In [ ]:
(self.bars.abs() >= 95).astype(int)

In [ ]:
v.default

In [ ]:
@pd.api.extensions.register_series_accessor("zscore")
@pd.api.extensions.register_dataframe_accessor("zscore")
class RollingZScoreZScore:
    def __init__(self, pandas_obj):
        self._obj = pandas_obj

    def __call__(self, *args, **kwargs):
        obj = self._obj
        ewm = kwargs.pop('ewm', False)
        rolling = kwargs.pop('rolling', False)
        accessor = 'rolling' if rolling else 'ewm' if ewm else None
        if accessor:
            accessor = getattr(obj, accessor)
            accessor_ = accessor(*args, **kwargs)
        else:
            accessor_ = obj
        return (obj - accessor_.mean()) / accessor_.std()

In [ ]:
self.bars.zscore(10, rolling=True)

In [ ]:
(self.bars - self. bars.rolling(10).mean()) / self.bars.rolling(10).std()

In [ ]:
hist = '5y'

In [ ]:
self = FutureStrategy(
    signal=MACDSignal(
        instrument=FutureFly.from_legs(
            ticker='ed',
            legs=[2, 6, 10],
            notional=1000,
            start_hist=hist
        ),
    ),

    trade_eop=False,
)

In [ ]:
self = FutureStrategy(
    signal=MACDSignal(
        instrument=FutureFly.from_legs(
            ticker='sfr',
            legs=[2, 6, 10],
            notional=1000,
            start_hist=hist
        ),
    ),
    # sl=0.1,
    # tp=0.1,
    trade_eop=False,
)

In [ ]:
def _any(n=1):
    pass
    


class EnsambleSignal:
        '''
    It has to be the same instrument
    methods:
        all
        any{n}
        mean
        average
    '''
    
    def __init__(self, signals, instrument=None, method=None):
        
        if instrument:
            for signal in signals:
                signal.set_instrument(instrument)
        self.signals = signals
        self.method = method
        
    def signal(self):
        if isinstance(method, str):
            if method[:3] == 'any':
                n = 1 if len(method) == 3 else n[3:]
                method = method[:3]
                
    @property
    def signals(self):
        return pd.concat([signal.signal for signal in self.signals], axis=1)
        

In [ ]:
self.pv.plot()

In [ ]:
check = pd.concat(
    [
        ff.pv.rename('pv'), 
        ff.pv.diff().rename('diff'), 
        ms.line.rename('line'),
        ms.signal.rename('signal'),
        pnl.rename('pnl'),
        t_costs.rename('tcosts'),
        ff.quote,
        ff.quote.diff(),
    ], 
    axis=1
)

In [ ]:
check.to_clipboard()

In [ ]:
ms.signal

In [ ]:
ms.trades

In [ ]:
ms.trades

In [ ]:
ms.line.to_clipboard()
ms.signal.to_clipboard()

In [ ]:
ms.signal.to_clipboard()

In [ ]:
ms.signal.fillna(0).diff().to_clipboard()

In [ ]:
ms.signal.diff().fillna(0).to_clipboard()

In [ ]:
ff.pv.to_clipboard()

In [ ]:
f = FutureChain('ty1')

In [ ]:
f.tick_value * f.notional

In [ ]:
FutureChain('tu1').tick_value

In [ ]:
_.to_clipboard()

In [ ]:
FutureChain('ty1').quote

In [ ]:
FutureChain('tu1').tick_size

In [ ]:
FutureChain('tu1').point_value / 100

In [ ]:
FutureChain('ty1').point_value / 100

In [ ]:
t = ba.getTable(
    namespace="BH_Query", 
    table="query_cont_bars", 
    symbol="EDc1", 
    bar_size="1day", 
    start_date=pd.to_datetime("2022-08-01"),
    roll_adjustment='multiplicative'
)

In [ ]:
catalog = ba.getCatalog()

In [ ]:
catalog

In [ ]:
ba.getTable("BH_SecMaster_Futures", 'roll_offset_mapping').to_clipboard()

In [ ]:
d = _

In [ ]:
d.loc[d['description'].str.contains('SOFR')]

In [ ]:
ba.getTable("BH_SecMaster_Futures", 'main', ticker='SFRU3')

In [ ]:
ba.getTable("BH_SecMaster_Futures", 'main', sector_sub_type='Index', rec_type='FUTURE')

In [ ]:
d = _

In [ ]:
s.trade

In [ ]:
trace1 = s1.pv.plot()

In [ ]:
trace1

In [ ]:
trace2 = swap_rate.plot()

In [ ]:
from plotly.subplots import make_subplots

# Create figure with secondary y-axis
fig = make_subplots(specs=[[{"secondary_y": True}]])

# Add traces
fig.add_trace(
    trace1.data[0],
    secondary_y=False,
)

fig.add_trace(
    trace2.data[0],
    secondary_y=True,
)